<a href="https://colab.research.google.com/github/opedoussaut/orbit/blob/main/ORBIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ORBIT
## From ontology to semantic contracts for LCA and EPD-ready reporting

**Version:** v1.0.0

This notebook presents the first stable release of the ORBIT semantic contract framework,
built on ORIONT and aligned with EF 3.1 midpoint impact categories.


## ORBIT v1.0.0 (Contract-based validation)

This version introduces semantic contracts for:
- LCA structural integrity
- LCIA metrics comparability
- Circularity assumptions
- Reporting and EPD readiness

Contracts are expressed as SHACL shapes and applied to RDF graphs
instantiated from LCA study data.

## How this notebook is structured

v1.0.0 is organized as a progression of contracts:

### 1) LCA-CORE (Study integrity)
Ensures the study has an explicit **goal/scope envelope**:
- Functional Unit
- Reference Flow
- System Boundary
- Activities with lifecycle stage + temporal validity

### 2) EF3.1 Metrics vocabulary (controlled metric identity)
Introduces a controlled list of LCIA metrics (EF3.1 midpoint) as **SKOS concepts**.

### 3) LCIA-METRICS (Comparability readiness)
Ensures each LCIA result declares:
- metric identity (from EF3.1 scheme)
- method binding (IRI)
- unit binding (IRI)
- numeric value
- scope target (product/stage/activity)

Each contract is demonstrated with:

✅ a conforming example  
❌ a non-conforming example  
and a SHACL conformance report.

In [20]:
!pip -q install rdflib pyshacl

from rdflib import Graph
from pyshacl import validate
import urllib.request

In [21]:
BRANCH = "v1.0.1"
BASE_RAW = f"https://raw.githubusercontent.com/opedoussaut/orbit/{BRANCH}"

CONTRACT_PATHS = {
    "lca_core": "contracts/lca-core.shacl.ttl",
    "lcia_metrics": "contracts/lcia-metrics.shacl.ttl",
    "reporting_epd": "contracts/reporting-epd.shacl.ttl",
}

VOCAB_PATHS = {
    "metrics": "vocab/metrics.skos.ttl",
}

EXAMPLE_PATHS = {
    "lca_ok":  "examples/lca-core_conforming.ttl",
    "lca_bad": "examples/lca-core_nonconforming.ttl",
    "lcia_ok":  "examples/lcia-metrics_conforming.ttl",
    "lcia_bad": "examples/lcia-metrics_nonconforming.ttl",
    "epd_ok":  "examples/epd_conforming.ttl",
    "epd_bad": "examples/epd_nonconforming.ttl",
}

In [22]:
def assert_url_ok(url: str):
    try:
        with urllib.request.urlopen(url) as _:
            return True
    except Exception as e:
        raise RuntimeError(f"Cannot load URL:\n{url}\n\n{e}")

def load_ttl_from_url(url: str) -> Graph:
    assert_url_ok(url)
    g = Graph()
    data = urllib.request.urlopen(url).read().decode("utf-8")
    g.parse(data=data, format="turtle")
    return g

def run_contract(data_graph: Graph, shacl_graph: Graph, label: str):
    conforms, report_graph, report_text = validate(
        data_graph=data_graph,
        shacl_graph=shacl_graph,
        inference="rdfs",
        abort_on_first=False
    )
    print(f"\n=== {label} ===")
    print("CONFORMS:", conforms)
    print(report_text)
    return conforms, report_text

## 1) LCA-CORE — Study integrity contract

This first contract validates that a study is structurally interpretable:
it must declare a functional unit, a reference flow, a system boundary, and at least one activity.
It also checks minimal temporal consistency for activities.

Goal: **fail fast** on incomplete or internally inconsistent studies before discussing metrics or reporting.

## 2) EF3.1 midpoint metrics vocabulary (controlled identity)

Before validating LCIA results, ORBIT introduces a controlled vocabulary of metrics.
This prevents “metric-by-string” ambiguity (e.g., “carbon footprint”, “climate change”, “GWP”)
and enables contracts to require that metric identity comes from the EF3.1 scheme.

In [23]:
lca_shacl = load_ttl_from_url(f"{BASE_RAW}/{CONTRACT_PATHS['lca_core']}")
lca_ok  = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lca_ok']}")
lca_bad = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lca_bad']}")

run_contract(lca_ok,  lca_shacl, "LCA-CORE / Conforming example")
run_contract(lca_bad, lca_shacl, "LCA-CORE / Non-conforming example")


=== LCA-CORE / Conforming example ===
CONFORMS: True
Validation Report
Conforms: True


=== LCA-CORE / Non-conforming example ===
CONFORMS: False
Validation Report
Conforms: False
Results (4):
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:message Literal("ReferenceFlow must declare refFlowUnit as an IRI (unit identifier).") ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:nodeKind sh:IRI ; sh:path orbit:refFlowUnit ]
	Focus Node: orbit:RF_Bad
	Result Path: orbit:refFlowUnit
	Message: ReferenceFlow must declare refFlowUnit as an IRI (unit identifier).
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:message Literal("ReferenceFlow must refer to a product/flow identifier (IRI).") ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:nodeKind sh:IRI ; sh:path orbit:refersTo 

(False,
 'Validation Report\nConforms: False\nResults (4):\nConstraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: [ sh:message Literal("ReferenceFlow must declare refFlowUnit as an IRI (unit identifier).") ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:nodeKind sh:IRI ; sh:path orbit:refFlowUnit ]\n\tFocus Node: orbit:RF_Bad\n\tResult Path: orbit:refFlowUnit\n\tMessage: ReferenceFlow must declare refFlowUnit as an IRI (unit identifier).\nConstraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: [ sh:message Literal("ReferenceFlow must refer to a product/flow identifier (IRI).") ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:nodeKind sh:IRI ; sh:path orbit:refersTo ]\n\tFocus Node: orbit:RF_Bad\n\tResult Path: orbit:refersTo\n\tMessage: ReferenceFlow must refer to a product/flow ide

In [24]:
metrics_vocab = load_ttl_from_url(f"{BASE_RAW}/{VOCAB_PATHS['metrics']}")
print("Triples in metrics vocabulary:", len(metrics_vocab))

Triples in metrics vocabulary: 113


## 3) LCIA-METRICS: Comparability readiness contract (EF3.1 aligned)

This contract does **not** compute LCIA.
It validates whether reported LCIA results are structurally comparable:

- metric identity is explicit and belongs to the EF3.1 scheme
- method is explicitly referenced (IRI)
- unit is explicitly referenced (IRI)
- numeric value exists
- scope target (appliesTo) is explicit and typed (product/stage/activity)

This makes “results” defensible artifacts rather than unqualified numbers.

In [25]:
lcia_shacl = load_ttl_from_url(f"{BASE_RAW}/{CONTRACT_PATHS['lcia_metrics']}")
lcia_ok  = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lcia_ok']}")
lcia_bad = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lcia_bad']}")

lcia_ok_all  = lcia_ok + metrics_vocab
lcia_bad_all = lcia_bad + metrics_vocab

run_contract(lcia_ok_all,  lcia_shacl, "LCIA-METRICS / EF3.1 / Conforming example")
run_contract(lcia_bad_all, lcia_shacl, "LCIA-METRICS / EF3.1 / Non-conforming example")


=== LCIA-METRICS / EF3.1 / Conforming example ===
CONFORMS: True
Validation Report
Conforms: True


=== LCIA-METRICS / EF3.1 / Non-conforming example ===
CONFORMS: False
Validation Report
Conforms: False
Results (3):
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:maxCount Literal("1", datatype=xsd:integer) ; sh:message Literal("LCIAResult must reference exactly one LCIA method IRI (orbit:hasMethod).") ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:nodeKind sh:IRI ; sh:path orbit:hasMethod ]
	Focus Node: orbit:BadR1
	Result Path: orbit:hasMethod
	Message: LCIAResult must reference exactly one LCIA method IRI (orbit:hasMethod).
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: orbit:LCIAResultShape_EF31
	Focus Node: orbit:BadR3
	Value Node: orbit:BadR3
	Source Constraint: [ rdf:typ

(False,
 'Validation Report\nConforms: False\nResults (3):\nConstraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: [ sh:maxCount Literal("1", datatype=xsd:integer) ; sh:message Literal("LCIAResult must reference exactly one LCIA method IRI (orbit:hasMethod).") ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:nodeKind sh:IRI ; sh:path orbit:hasMethod ]\n\tFocus Node: orbit:BadR1\n\tResult Path: orbit:hasMethod\n\tMessage: LCIAResult must reference exactly one LCIA method IRI (orbit:hasMethod).\nConstraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: orbit:LCIAResultShape_EF31\n\tFocus Node: orbit:BadR3\n\tValue Node: orbit:BadR3\n\tSource Constraint: [ rdf:type sh:SPARQLConstraint ; sh:message Literal("orbit:appliesTo must be typed as orbit:ProductSystem or orbit:LCStage or orbit:Activity.") ; 

## 4) REPORTING-EPD — EPD readiness contract

This pack validates the *published reporting envelope*:
- PCR reference + standard reference + publication metadata
- verification status declared
- explicit boundary declaration at report level
- required EF3.1 metric subset is present on the referenced ProductSystem

This does not certify an EPD. It makes an EPD-style report **structurally defensible and checkable**.

In [26]:
epd_shacl = load_ttl_from_url(f"{BASE_RAW}/{CONTRACT_PATHS['reporting_epd']}")

epd_ok  = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['epd_ok']}")
epd_bad = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['epd_bad']}")

# Merge EF3.1 metrics vocabulary so scheme references are resolvable
epd_ok_all  = epd_ok + metrics_vocab
epd_bad_all = epd_bad + metrics_vocab

run_contract(epd_ok_all,  epd_shacl, "REPORTING-EPD / Conforming example")
run_contract(epd_bad_all, epd_shacl, "REPORTING-EPD / Non-conforming example")


=== REPORTING-EPD / Conforming example ===
CONFORMS: False
Validation Failure - A SPARQL Constraint must not contain a VALUES clause.

=== REPORTING-EPD / Non-conforming example ===
CONFORMS: False
Validation Failure - A SPARQL Constraint must not contain a VALUES clause.


(False,
 'Validation Failure - A SPARQL Constraint must not contain a VALUES clause.')

## Conclusion — From ontology to semantic contracts

ORBIT v1.0.0 demonstrates how an existing, well-founded ontology (ORIONT) can be turned into a **usable reference structure** through semantic contracts.

By combining:
- an ORIONT-aligned structural model (LCA-CORE),
- a controlled EF3.1 metric vocabulary (SKOS),
- contract-based validation of LCIA results (LCIA-METRICS),
- and an explicit reporting envelope for EPD-style claims (REPORTING-EPD),

this version shifts the focus from *defining concepts* to *making assumptions, boundaries, and reporting obligations explicit and checkable*.

The contracts introduced here do not compute results, certify studies, or replace standards. Instead, they operate at a critical intermediate layer — between methodology and implementation — where many comparability, governance, and credibility issues typically arise.

In this sense, ORBIT v1.0.0 establishes a foundation for:
- structurally defensible LCA models,
- comparable and interpretable LCIA results,
- and machine-checkable readiness for standardized reporting (e.g. EPDs).

Future versions will focus on stabilizing these contracts, refining vocabularies, and exploring how such semantic contracts can support scalable industrial workflows, platform integration, and assisted model construction, while preserving methodological rigor.

ORBIT thus positions semantic contracts as a practical bridge between life-cycle assessment theory and operational sustainability decision-making.